<a href="https://colab.research.google.com/github/LeonMilosevic/covid_19_analysis/blob/main/covid_19_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Covid-19 EDA**

## Introduction



1. We will explore how Asian countries dealt with pandemic compared to Europen countries. 
2. We will explore specifically how South Korea battled with the pandemic.
3. We will create a model to distinguish released and isolated patients. 

### Datasets used:


*   https://www.kaggle.com/sudalairajkumar/novel-corona-virus-2019-dataset
*   https://www.kaggle.com/kimjihoo/coronavirusdataset/
*   https://www.kaggle.com/dumbgeek/countries-dataset-2020
*   https://www.kaggle.com/eng0mohamed0nabil/population-by-country-2020